# Assignment 3 (15 points)
(due on Nov. 25, 23:59pm)

In the third assignment, we will write a Convolutional Neural Network (CNN) together with training and evaluation routines.

The task description can be found [below](#Task).

**Important**: I strongly recommend to use *Google Collab (GC)* for this assignment. Make yourself familiar with running Jupyter notebooks on GC (especially selecting the right runtime, i.e., Python 3 + GPU). This will make your life a lot easier, as training will be faster and you can easily debug problems in your model.

In [ ]:
import torch
import torchvision
from torchvision.datasets import MNIST, CIFAR10
import torchvision.transforms as transforms
from torch.utils.data.dataset import Subset
from torch.utils.data import DataLoader

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from sklearn.model_selection import StratifiedShuffleSplit

torch.manual_seed(1234);
np.random.seed(1234);

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])
         
ds_train = CIFAR10('/tmp/cifar', 
                 train=True, 
                 transform=transforms.ToTensor(), 
                 target_transform=None, 
                 download=True)

ds_test = CIFAR10('/tmp/cifar', 
                 train=False, 
                 transform=transforms.ToTensor(), 
                 target_transform=None, 
                 download=True)

lab = [ds_train[x][1] for x in range(len(ds_train))]

In [ ]:
def generate_train_indices(n_splits, train_size, lab):
    s = StratifiedShuffleSplit(
        n_splits=n_splits, 
        train_size=train_size, 
        test_size=None)
    
    return [i.tolist() for i, _ in s.split(lab, lab)]

train_indices = generate_train_indices(10, 500, lab)

In [ ]:
from collections import Counter
Counter(lab)

In [ ]:
classes = ['plane', 
           'car', 
           'bird', 
           'cat',
           'deer', 
           'dog', 
           'frog', 
           'horse', 
           'ship', 
           'truck']

In [ ]:
ds_train_subset = Subset(ds_train, train_indices[1])

In [ ]:
from collections import Counter
print(Counter([ds_train_subset[i][1] for i in range(len(ds_train_subset))]))

In [ ]:
def show_images(ds: torchvision.datasets.cifar.CIFAR10, 
                indices: list):
    
    assert np.max(indices) < len(ds)
    
    plt.figure(figsize=(9, len(indices)));
    for j,idx in enumerate(indices):
        plt.subplot(1,len(indices),j+1)
        plt.imshow(ds[idx][0].permute(1,2,0).numpy())
        plt.title('Label={}'.format(classes[ds[idx][1]]),fontsize=9)

show_images(ds_train_subset, [1,3,499,2,10])

In [ ]:
def count_params(net):
    num_param = 0
    for p in net.parameters():
        num_param += p.numel()
    return num_param

### Task

The assignment is split into 4 parts: (1) writing the model definition, (2) writing the training code, (3) writing the testing code and (4) writing the *glue* code which initializes the model, sets the optimizer (and scheduler) and eventually performs a couple of epochs of training + testing.

*First*, implement the following **convolutional neural network (CNN)**: It consists of 3 blocks and a simple linear classifier at the end.

My notation denotes the following:

- `Conv2D(in_channels, out_channels, kernel_size, padding)` - 2D Convolution
- `MaxPool(kernel_size, stride, padding)` - Max. pooling
- `AvgPool(kernel_size, stride, padding)` - Avg. pooling
- `Dropout(dropout_probability)` - Dropout layer
- `BatchNorm2D` - 2D batch normalization

All these operations can also be found in the [PyTorch documentaton](https://pytorch.org/docs/stable/index.html).

**Block 1**

```
Conv2D(  3,128,3,1) -> Batchnorm2D -> LeakyReLU(0.1)
Conv2D(128,128,3,1) -> Batchnorm2D -> LeakyReLU(0.1)
Conv2D(128,128,3,1) -> Batchnorm2D -> LeakyReLU(0.1)
MaxPool(2,2,0)
Dropout(0.5)
```
The output size at that point should be $N \times 128 \times 16 \times 16$.

**Block 2**

```
Conv2D(128,256,3,1) -> Batchnorm2D -> LeakyReLU(0.1)
Conv2D(256,256,3,1) -> Batchnorm2D -> LeakyReLU(0.1)
Conv2D(256,256,3,1) -> Batchnorm2D -> LeakyReLU(0.1)
MaxPool(2,2,0)
Dropout(0.5)
```
The output size at that point should be $N \times 128 \times 8 \times 8$.

**Block 3**

```
Conv2D(256,512,3,0) -> Batchnorm2D -> LeakyReLU(0.1)
Conv2D(512,256,1,0) -> Batchnorm2D -> LeakyReLU(0.1)
Conv2D(256,128,1,0) -> Batchnorm2D -> LeakyReLU(0.1)
AvgPool(6,2,0)
Dropout(0.5)
```
The output size at that point should be $N \times 128 \times 1 \times 1$.

**Classifier**

View the output of the last block as a $1 \times 128$ tensor and add a 
linear layer mapping from $\mathbb{R}^{128} \rightarrow \mathbb{R}^{10}$
(include bias).

In [ ]:
class ConvNet(nn.Module): 
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        
        def make_block(conv_config, pooling_op=None, use_dropout=False):
            mlist = nn.ModuleList()
            for in_c, out_c, k_size, pad in conv_config:
                mlist.extend([
                    nn.Conv2d(in_c, out_c, k_size, padding=pad),
                    nn.BatchNorm2d(out_c),
                    nn.LeakyReLU(0.1)
                ])
            mlist.append(pooling_op)
            if use_dropout:
                mlist.append(nn.Dropout(0.5))
            return mlist

        self.block1 = make_block([
            [  3,128,3,1],
            [128,128,3,1],
            [128,128,3,1]], 
            nn.MaxPool2d(2,stride=2,padding=0), 
            use_dropout=True)
        
        self.block2 = make_block([
            [128,256,3,1],
            [256,256,3,1],
            [256,256,3,1]], 
            nn.MaxPool2d(2,stride=2,padding=0),
            use_dropout=True)

        self.block3 = make_block([
            [256,512,3,0],
            [512,256,1,0],
            [256,128,1,0]], 
            nn.AvgPool2d(6,stride=2,padding=0),
            use_dropout=False)
        
        self.classifier = nn.Linear(128,10)
    
    def forward(self, x):
        for l in self.block1: x = l(x)
        for l in self.block2: x = l(x)
        for l in self.block3: x = l(x)
        x = x.view(x.size(0),-1)
        x = self.classifier(x)
        return x

Test your implementation with

In [ ]:
net = ConvNet(10)
out = net(torch.rand(5,3,32,32))
print(out.size())
print(count_params(net))

**Second**, write a training routine

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    
    model.train()
    
    epoch_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()

    print('Train Epoch: {} \tLoss: {:.6f}'.format(epoch, epoch_loss))

**Third**, write a testing routine ...

In [ ]:
def test(model, device, test_loader):
    model.eval()
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

**Finally**, write the *glue* code which performs a couple of epochs (e.g., 100) of training and, after each epoch, test the model.

In [ ]:
torch.manual_seed(1234)

device = 'cpu'

train_loader = torch.utils.data.DataLoader(
    ds_train_subset,
    batch_size=32,
    shuffle=True)

test_loader = torch.utils.data.DataLoader(
    ds_test, 
    batch_size=64, 
    shuffle=False)

model = ConvNet().to(device)

optimizer = optim.SGD(
    model.parameters(), 
    lr=0.01, 
    momentum=0.9)

for epoch in range(1,10+1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)